In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, auc, roc_curve
import tensorflow as tf

In [ ]:
!pip install kaleido

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = os.path.abspath("./drive/MyDrive/MegaClassifier")

In [ ]:
MODEL_NAME = "MegaClassifier_a"
VERSION = "v1"

In [ ]:
DATASETS = {
    "MegaClassifier_a": "onlyDetectionsForTrain",
    "MegaClassifier_b": "emptyOriginalAnimalDetection",
    "MegaClassifier_c": "emptyNonEmptyDataset",
}

VERSIONS = {
    "v1": [
        "v1.0",
        "v1.1",
        "v1.2",
        "v1.3",
    ],
    "v2": [
        "v2.0",
        "v2.1",
        "v2.2"
    ],
    "v3": [
        "v3.0",
        "v3.1",
    ],
    "v4": [
        "v4.0.0",
        "v4.0.1",
        "v4.0.2",
        "v4.1.0",
        "v4.1.1",
        "v4.1.2",
        "v4.2.0",
        "v4.2.1",
        "v4.2.2",
    ]
}

In [ ]:
DATASET_PATH_ZIP = os.path.join(BASE_PATH, f"data/processed/{DATASETS[MODEL_NAME]}.zip")
with zipfile.ZipFile(DATASET_PATH_ZIP, 'r') as zip_ref:
      zip_ref.extractall("./data/processed")
DATASET_DIR = os.path.abspath(f"data/processed/{DATASETS[MODEL_NAME]}")

In [ ]:
IMAGE_SIZE = (456, 456)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)
test_images = datagen.flow_from_directory(
  directory=f"{DATASET_DIR}/test",
  classes=['vacia', 'animal'],
  target_size=IMAGE_SIZE,
  class_mode="binary",
  shuffle=False,
  seed=SEED,
)

In [ ]:
test_images.class_indices

In [ ]:
def create_confusion_matrix(conf_matrix, MODEL_NAME, SUBVERSION):
  fig = go.Figure(
    data=go.Heatmap(
        z=conf_matrix,
        x=["Animal", "Empty"],
        y=["Empty", "Animal"],
        text=conf_matrix,
        texttemplate="%{text}",
        colorscale="Blues",
        showscale=True,
    )
  )


  fig.update_layout(
      title=f"Confusion Matrix {MODEL_NAME} {SUBVERSION}",
      xaxis_title="Predicted value",
      yaxis_title="Actual value",
      xaxis=dict(tickmode="array", tickvals=[0, 1], ticktext=["Animal", "Empty"]),
      yaxis=dict(tickmode="array", tickvals=[0, 1], ticktext=["Empty", "Animal"]),
      template="seaborn",
      width=600,
      height=600,
  )

  return fig

def create_roc_curve(fpr, tpr, roc_auc, MODEL_NAME, SUBVERSION):
  fig = go.Figure()

  fig.add_trace(
      go.Scatter(
          x=fpr, y=tpr, mode="lines", name=f"(AUC = {roc_auc:.4f})"
      )
  )

  fig.add_trace(
      go.Scatter(
          x=[0, 1],
          y=[0, 1],
          mode="lines",
          name="(AUC = 0.5000)",
          line=dict(dash="dash"),
      )
  )

  fig.update_layout(
      title=f"ROC Curve {MODEL_NAME} {SUBVERSION}",
      xaxis_title="False Positive Rate",
      yaxis_title="True Positive Rate",
      legend_title="Curves",
      template="seaborn",
      width=700,
      height=500,
      xaxis=dict(
          tickmode="linear",
          tick0=0,
          dtick=0.1,
      ),
  )

  return fig

In [ ]:
for SUBVERSION in VERSIONS[VERSION]:

  SUBMODEL = SUBVERSION.replace(".", "_")

  LOGS_PATH = os.path.join(BASE_PATH, f"logs/{MODEL_NAME}/{VERSION}/{SUBVERSION}")
  MODELS_PATH = os.path.join(BASE_PATH, f"models/{MODEL_NAME}/{VERSION}")

  REPORTS_PATH = os.path.join(BASE_PATH, f"reports/3_evaluation_results/{MODEL_NAME}/{VERSION}/")
  os.makedirs(REPORTS_PATH, exist_ok=True)

  model = tf.keras.models.load_model(os.path.join(MODELS_PATH, f"{MODEL_NAME}_{SUBMODEL}.keras"))

  y_pred_prob = model.predict(test_images)
  y_true = test_images.labels

  y_pred_class = (y_pred_prob > 0.5).astype(int)
  conf_matrix = confusion_matrix(y_true, y_pred_class)

  FP = conf_matrix[0][0]
  TN = conf_matrix[0][1]
  TP = conf_matrix[1][0]
  FN = conf_matrix[1][1]

  accuracy_predict = (TP + TN) / (TP + TN + FP + FN)
  recall_predict = TP / (TP + FN)
  specificity_predict = TN / (TN + FP)
  precision_predict = TP / (TP + FP)
  f1_score_predict = 2 * (precision_predict * recall_predict) / (precision_predict + recall_predict)

  fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
  auc_predict = auc(fpr, tpr)

  confusion_matrix_fig = create_confusion_matrix(conf_matrix, MODEL_NAME, SUBVERSION)
  confusion_matrix_fig.write_image(os.path.join(REPORTS_PATH, f"confusion_matrix_{SUBVERSION}.png"))

  roc_curve_fig = create_roc_curve(fpr, tpr, auc_predict, MODEL_NAME, SUBVERSION)
  roc_curve_fig.write_image(os.path.join(REPORTS_PATH, f"roc_curve_{SUBVERSION}.png"))

  predicted_results = pd.DataFrame(
      {
          "accuracy_predict": [accuracy_predict],
          "precision_predict": [precision_predict],
          "recall_predict": [recall_predict],
          "specificity_predict": [specificity_predict],
          "auc_predict": [auc_predict],
          "f1_score_predict": [f1_score_predict],
      }
  )
  predicted_results.to_csv(os.path.join(LOGS_PATH, f"predicted_results_{SUBVERSION}.csv"), index=False, sep=";")

  results = model.evaluate(test_images)
  metric_names = [
      "test_loss",
      "test_accuracy",
      "test_precision",
      "test_recall",
      "test_auc",
  ]
  evaluation_results = {name: value for name, value in zip(metric_names, results)}

  evaluation_results['test_f1_score'] = 2 * (evaluation_results['test_precision'] * evaluation_results['test_recall']) / (evaluation_results['test_precision'] + evaluation_results['test_recall'])

  evaluation = pd.DataFrame([evaluation_results])
  evaluation.to_csv(os.path.join(LOGS_PATH, f"evaluation_results_{SUBVERSION}.csv"), index=False, sep=";")

  confusion_matrix_fig.show()
  print()
  roc_curve_fig.show()
  print(predicted_results)
  print()
  print(evaluation)
  print()
  print()
